In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime
import calendar
from datetime import timedelta
import datetime as dt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

In [ ]:
from datetime import datetime
import calendar
from datetime import timedelta
import datetime as dt

In [ ]:
# train data consists of 55M rows, taking 400K rows initially
train = pd.read_csv("/kaggle/input/iiitb2019nyctaxifare/TrainTest/train.csv", nrows = 2000000)
test = pd.read_csv("/kaggle/input/iiitb2019nyctaxifare/TrainTest/test.csv")

In [ ]:
train.info()

In [ ]:
test.head()
test.info()

In [ ]:
# change the format of datetime
def change_datetime_format(data):
    data['pickup_datetime']=pd.to_datetime(data['pickup_datetime'],format='%Y-%m-%d %H:%M:%S UTC')
    return data

In [ ]:
# split the datetime
def split_datetime(data):
    data['pickup_date']= data['pickup_datetime'].dt.date
    data['pickup_day']=data['pickup_datetime'].apply(lambda x:x.day)
    data['pickup_hour']=data['pickup_datetime'].apply(lambda x:x.hour)
    data['pickup_day_of_week']=data['pickup_datetime'].apply(lambda x:calendar.day_name[x.weekday()])
    data['pickup_month']=data['pickup_datetime'].apply(lambda x:x.month)
    data['pickup_year']=data['pickup_datetime'].apply(lambda x:x.year)
    return data

In [ ]:
# removing outliers in latitude and longitude
def remove_outliers(data):
    
    misplaced_locations_index = []

    for i, val in enumerate(zip(data.pickup_latitude,data.dropoff_latitude,data.pickup_longitude,data.dropoff_longitude)):

        #print(val)
        #break

        lat1,lat2,lon1,lon2 = val
        #co_ords1 = (lat1, lon1)
        #co_ords2 = (lat2, lon2)

        if lat1 < 40.5 or lat1 > 41.8 or lat2 < 40.5 or lat2 > 41.8 or lon1 < -74.5 or lon1 > -72.8 or lon2 < -74.5 or lon2 > -72.8:
            misplaced_locations_index.append(i)


    data = data.drop(misplaced_locations_index)
    return data

In [ ]:
def remove_null(data):
    data = data.dropna()
    return data

In [ ]:
def remove_neg_pass_count(data):
    data = data.drop(data[data['passenger_count'] <= 0 ].index.tolist())
    return data

In [ ]:
def remove_neg_fare_amount(data):
    data = data.drop(data[data['fare_amount'] <= 0].index.tolist())
    data = data[data['fare_amount'] <=8]
    return data

In [ ]:
#calculate trip distance in miles
def distance(lat1, lat2, lon1,lon2):
    p = 0.017453292519943295 # Pi/180
    a = 0.5 - np.cos((lat2 - lat1) * p)/2 + np.cos(lat1 * p) * np.cos(lat2 * p) * (1 - np.cos((lon2 - lon1) * p)) / 2
    return 0.6213712 * 12742 * np.arcsin(np.sqrt(a))

In [ ]:
def calc_distance(data):
    data['trip_distance']=data.apply(lambda row:distance(row['pickup_latitude'],row['dropoff_latitude'],row['pickup_longitude'],row['dropoff_longitude']),axis=1)
    return data

In [ ]:
# Let us encode day of the week to numbers
def encodeDays(day_of_week):
    day_dict={'Sunday':0,'Monday':1,'Tuesday':2,'Wednesday':3,'Thursday':4,'Friday':5,'Saturday':6}
    return day_dict[day_of_week]

In [ ]:
def pickupday_encode(data):
    data['pickup_day_of_week']=data['pickup_day_of_week'].apply(lambda x:encodeDays(x))
    return data

In [ ]:
def drop_columns(data):
    data.drop(columns=['key','pickup_datetime','pickup_date'], inplace=True)
    return data

In [ ]:
# clean the data
def cleandata(data,flag = True):
    if flag == True:
        data = change_datetime_format(data)
        data = split_datetime(data)
        data = remove_outliers(data)
        data = remove_null(data)
        data = remove_neg_pass_count(data)
        data = remove_neg_fare_amount(data)
        data = calc_distance(data)
        data = pickupday_encode(data)
        data = drop_columns(data)
    else:
        data = change_datetime_format(data)
        data = split_datetime(data)
        data = pickupday_encode(data)
        data = calc_distance(data)
    return data

In [ ]:
train = cleandata(train)
test = cleandata(test,False)

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
test.info()


In [ ]:
def processDataForModelling(data,target,is_train=True,split=0.3):
    data_1=data
    # One hot Encoding
    data_1=pd.get_dummies(data_1)
    if is_train==True:
        X=data_1.drop([target],axis=1)
        y=data_1[target]
        X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=split,random_state=123)
        
        print("Shape of Training Features",X_train.shape)
        print("Shape of Validation Features ",X_test.shape)
        
        return X_train, X_test, y_train, y_test
    else:
        print ("Shape of Test Data",data_1.shape)
        return data_1

In [ ]:
X_train, X_test, y_train, y_test=processDataForModelling(train,'fare_amount',is_train=True,split=0.2)

In [ ]:
#avg_fare=round(np.mean(y_train),2)
#avg_fare

In [ ]:
# Baseline Model
#baseline_pred=np.repeat(avg_fare,test.shape[0])

In [ ]:
"""p = pd.DataFrame(baseline_pred)
datasets = pd.concat([test['key'],p[0]],axis=1)
datasets.columns = ['key', 'fare_amount']
datasets.to_csv('submission1.csv',index = False)
"""

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
key_value = test['key']

In [ ]:
test = test.drop(columns = ['pickup_date', 'key','pickup_datetime'])

In [ ]:
test.fillna(method = 'ffill',inplace = True)

In [ ]:
#for i in test.columns:
 #   print(i,test[i].isnull().sum())

In [ ]:
#linearRegressor = LinearRegression()
#linearRegressor.fit(X_train, y_train)

In [ ]:
#y_pred = linearRegressor.predict(test)

In [ ]:
'''p = pd.DataFrame(y_pred)
datasets = pd.concat([key_value,p[0]],axis=1)
datasets.columns = ['key', 'fare_amount']
datasets.to_csv('submission2.csv',index = False)
'''

In [ ]:
'''from IPython.display import FileLink
FileLink(r'submission2.csv')
'''

In [ ]:
#sub = pd.read_csv("/kaggle/working/submission2.csv")

In [ ]:
#sub.shape

In [ ]:
#sub.info()

In [ ]:
#sub.describe()

In [ ]:
#sub.isnull().sum()

In [ ]:
import lightgbm as lgb
train_data=lgb.Dataset(X_train,label=y_train)
param = {'num_leaves':31, 'num_trees':5000,'objective':'regression'}
param['metric'] = 'l2_root'
num_round=5000
cv_results = lgb.cv(param, train_data, num_boost_round=num_round, nfold=10,verbose_eval=20, early_stopping_rounds=20,stratified=False)
lgb_bst=lgb.train(param,train_data,len(cv_results['rmse-mean']))
lgb_pred = lgb_bst.predict(X_test)
lgb_rmse=np.sqrt(mean_squared_error(lgb_pred, y_test))
print("RMSE for Light GBM is ",lgb_rmse)

In [ ]:
lgb_pred = lgb_bst.predict(test)

In [ ]:
p = pd.DataFrame(lgb_pred)
datasets = pd.concat([key_value,p[0]],axis=1)
datasets.columns = ['key', 'fare_amount']
datasets.to_csv('submission4.csv',index = False)

In [ ]:
from IPython.display import FileLink
FileLink(r'submission4.csv')